In [30]:
!pip install -U tensorflow-addons

Requirement already up-to-date: tensorflow-addons in /usr/local/lib/python3.7/dist-packages (0.13.0)


In [31]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa


In [32]:
from tensorflow.keras import layers,models
from tensorflow_addons.layers import SpatialPyramidPooling2D 
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dropout, Flatten, Dense, Reshape

In [43]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [44]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [45]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [47]:
model = tf.keras.models.Sequential([
  tfa.layers.SpatialPyramidPooling2D([1,4,12] ),
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
#model.summary()
model.fit(
    ds_train,
    epochs=50,
    validation_data=ds_test,
)
model.summary()

Epoch 1/50
469/469 [==============================] - 3s 6ms/step - loss: 0.5530 - sparse_categorical_accuracy: 0.8576 - val_loss: 0.2767 - val_sparse_categorical_accuracy: 0.9240
Epoch 2/50
469/469 [==============================] - 3s 7ms/step - loss: 0.2539 - sparse_categorical_accuracy: 0.9283 - val_loss: 0.2095 - val_sparse_categorical_accuracy: 0.9392
Epoch 3/50
469/469 [==============================] - 5s 11ms/step - loss: 0.2000 - sparse_categorical_accuracy: 0.9431 - val_loss: 0.1749 - val_sparse_categorical_accuracy: 0.9505
Epoch 4/50
469/469 [==============================] - 4s 8ms/step - loss: 0.1668 - sparse_categorical_accuracy: 0.9521 - val_loss: 0.1506 - val_sparse_categorical_accuracy: 0.9563
Epoch 5/50
469/469 [==============================] - 5s 10ms/step - loss: 0.1428 - sparse_categorical_accuracy: 0.9586 - val_loss: 0.1328 - val_sparse_categorical_accuracy: 0.9614
Epoch 6/50
469/469 [==============================] - 5s 11ms/step - loss: 0.1256 - sparse_categor

correct one.